In [1]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [5]:
# Vehicles Available: Table B08141
veh_variables = {
    'B08141_001E': 'total',
    'B08141_001M': 'total_moe',
    'B08141_002E': 'no_vehicle',
    'B08141_002M': 'no_vehicle_moe',
    'B08141_003E': 'one_vehicle',
    'B08141_003M': 'one_vehicle_moe',
    'B08141_004E': 'two_vehicle',
    'B08141_004M': 'two_vehicle_moe',
    'B08141_005E': 'three_vehicle',
    'B08141_005M': 'three_vehicle_moe',
}
veh_columns_out = [
    'total', 'total_moe',
    'pct_no_vehicle', 'pct_no_vehicle_moe',
    'pct_one_vehicle', 'pct_one_vehicle_moe',
    'pct_two_or_more', 'pct_two_or_more_moe'
]

In [3]:
def combine_vehicles(in_df):
    '''
    Outputs: simplified vehicle breakdown with MOEs
    '''
    df = in_df.copy()
    
    ### AGGREGATE ESTIMATES
    # Define a list of columns to combine
    two_more_cols = ['two_vehicle', 'three_vehicle']
    df['two_or_more'] = df[two_more_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    two_more_moes = [f'{col}_moe' for col in two_more_cols]    
    df['two_or_more_moe'] = (df[two_more_moes]**2).sum(axis='columns')**0.5
    
    for group in ['no_vehicle', 'one_vehicle', 'two_or_more']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df['total'] < 25, f'pct_{group}'] = float('NaN')
        df.loc[df['total'] < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')
    
    return df

In [11]:
def combine_tracts(in_df):
    '''

    Outputs:
    A modified version of in_df with census tracts combined
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df_copy = in_df.copy()
    df = pd.DataFrame(columns=veh_variables.values())

    ### CLEAN UNUSUAL MOES
    df_copy = df_copy.replace(-555555555.0, 0)

    for c in veh_variables.values():
        if "moe" not in c:
            # sum the totals
            df.at[0, c] = df_copy[c].sum()
        else:
            # sum of squares the moes
            df.at[0, c] = np.sqrt((df_copy[c]**2).sum())

    df_out = combine_vehicles(df)
    
    return df_out

In [7]:
def get_vehicle_precombo(year_in, place_num):
    # for getting mode for census tracts before they are combined
    df = pd.DataFrame(
        c.acs5.get(
            list(veh_variables.keys()),
            {'for': place_num, 'in': 'state:06 county:013'},
            year=year_in
        )
    )
    df = df.rename(columns=veh_variables)
    df = df.drop(columns=["state", "county", "tract"])
    
    return df

In [23]:
def get_city_df(year_in):
    # City
    df = pd.DataFrame(
        c.acs5.get(
            list(veh_variables.keys()),
            {'for': 'place:60620', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=veh_variables)
    df_out = combine_vehicles(df)

    return df_out

In [19]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 for select tracts around the BART station
# county:013
# city: 60620
c_tracts = 'tract:375000, 376000, 377000, 374000, 381000'

df_tracts_2014 = get_vehicle_precombo(2014, c_tracts)
df_tracts_2019 = get_vehicle_precombo(2019, c_tracts)

In [20]:
df_comb_2014 = combine_tracts(df_tracts_2014)
df_tracts_out_2014 = df_comb_2014[veh_columns_out]
df_tracts_out_2014.insert(0, "year", 2014)
df_comb_2019 = combine_tracts(df_tracts_2019)
df_tracts_out_2019 = df_comb_2019[veh_columns_out]
df_tracts_out_2019.insert(0, "year", 2019)

In [24]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 in Richmond City
df_city_2014 = get_city_df(2014)
df_city_2019 = get_city_df(2019)

In [25]:
df_city_out_2014 = df_city_2014[veh_columns_out]
df_city_out_2014.insert(0, "year", 2014)
df_city_out_2019 = df_city_2019[veh_columns_out]
df_city_out_2019.insert(0, "year", 2019)

In [26]:
#export settings, for a combined csv
df_tracts_out = pd.concat([df_tracts_out_2014, df_tracts_out_2019])
df_city_out = pd.concat([df_city_out_2014, df_city_out_2019])

In [27]:
df_city_out

,year,total,total_moe,pct_no_vehicle,pct_no_vehicle_moe,pct_one_vehicle,pct_one_vehicle_moe,pct_two_or_more,pct_two_or_more_moe
0,2014,46973.0,1272.0,0.04196,0.008740,0.244821,0.018180,0.713218,0.032811
0,2019,52366.0,1212.0,0.03340,0.009114,0.237807,0.018146,0.728793,0.035268
